In [1]:
# public packages
import andes
import ams
import cvxpy as cp
import numpy as np
import gurobipy

# v0: make sure the modifed model based on 123 system is consistent with the original model in ams

# Para preparetion

In [3]:
# MD parameters
vsg_num = 8

lb_M = np.array([0, 0, 0, 0, 0, 0, 0, 0])
ub_M = np.array([6, 6, 6, 6, 6, 6, 6, 6])

lb_D = np.array([0, 0, 0, 0, 0, 0, 0, 0])
ub_D = np.array([4, 4, 4, 4, 4, 4, 4, 4])

# Load ams model

In [5]:
ssp = ams.load('ams_case123_REGCV1_v5.xlsx')

ssp.LDOPF.setup()

ssp.LDOPF.om.mdl.solve()

0.4008419375980871

In [6]:
ssp.LDOPF.obj.v

0.4008419375980871

In [7]:
ssp.LDOPF.om.vars

OrderedDict([('pg', Variable((11,))),
             ('pn', Variable((117,))),
             ('qg', Variable((11,))),
             ('qn', Variable((117,))),
             ('vsq', Variable((117,))),
             ('pl', Variable((116,))),
             ('ql', Variable((116,)))])

In [8]:
pg_ams = ssp.LDOPF.om.pg.value

pg_ams

array([0.02001194, 0.00001195, 0.00001201, 0.05448066, 0.07447504,
       0.0340014 , 0.0340014 , 0.0340014 , 0.0340014 , 0.0340014 ,
       0.0340014 ])

In [9]:
ssp.LDOPF.pg.get_idx()

['PV_1',
 'PV_10',
 'PV_11',
 'PV_2',
 'PV_3',
 'PV_4',
 'PV_5',
 'PV_6',
 'PV_7',
 'PV_8',
 'PV_9']

# Define new variables, constraints, and obj

In [10]:
# M and D

M = cp.Variable(vsg_num, name="M")
D = cp.Variable(vsg_num, name="D")

constraints_M = [M >= lb_M, M <= ub_M]
constraints_D = [D >= lb_D, D <= ub_D]

In [11]:
constriants_ams = [ssp.LDOPF.om.constrs[cname] for cname in ssp.LDOPF.om.constrs.keys()]

In [12]:
constraints = constriants_ams + constraints_M + constraints_D

# Formulate new problem

pg = ['PV_1',
 'PV_10',
 'PV_11',
 'PV_2',
 'PV_3',
 'PV_4',
 'PV_5',
 'PV_6',
 'PV_7',
 'PV_8',
 'PV_9']

 SG: PV_1, PV_2, PV_3 (slack bus)

 IBR: PV_4, PV_5, ..., PV_11

In [13]:
# Cost data

c2 = np.array([0.01, 0, 0, 0.01, 0.02, 0, 0, 0, 0, 0, 0]) # gen quadratic cost
c1 = np.array([  20, 0, 0,   20,   30, 0, 0, 0, 0, 0, 0]) # gen linear cost

# c2 = np.array([0.01, 0.01, 0.02, 0, 0, 0, 0, 0, 0, 0, 0]) # gen quadratic cost
# c1 = np.array([  20,   20,   30, 0, 0, 0, 0, 0, 0, 0, 0]) # gen linear cost

d  = np.array([0, 0, 0, 0, 0, 0, 0, 0])                   # gen virtual inertia cost

pg = ssp.LDOPF.om.pg

# pg_power = cp.power(pg, 2)
pg_power = pg**2

cost = cp.sum(c2 @ pg_power + c1 @ pg + d @ M)

obj = cp.Minimize(cost)

In [14]:
ssp.LDOPF.c2.v

array([0.01, 0.01, 0.02, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ])

In [15]:
# Create the optimization problem
prob = cp.Problem(obj, constraints)

In [16]:
# built-in solver: ECOS_BB
prob.solve(solver=cp.ECOS_BB, verbose=True)

                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Sep 06 09:07:36 PM: Your problem has 621 variables, 17 constraints, and 0 parameters.
(CVXPY) Sep 06 09:07:36 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Sep 06 09:07:36 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Sep 06 09:07:36 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Sep 06 09:07:36 PM: Compiling problem (target solver=ECOS_BB).
(CVXPY) Sep 06 09:07:36 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffi

2.0000400000025937

In [18]:
# Solve the optimization problem

# prob.solve(solver=cp.SCIP, verbose=True)

prob.solve()


2.00066158718484

In [20]:
# Solve the optimization problem

# prob.solve(solver=cp.GUROBI, verbose=True)

prob.solve()


2.00066158718484

In [21]:
ssp.LDOPF.c1

RParam: c1 <GCost>, v=[20. 20. 30.  0.  0.  0.  0.  0.  0.  0.  0.]

In [22]:
ssp.LDOPF.c1.owner

GCost (11 devices) at 0x7f727a312550

In [23]:
ssp.LDOPF.c2

RParam: c2 <GCost>, v=[0.01 0.01 0.02 0.   0.   0.   0.   0.   0.   0.   0.  ]

In [24]:
ssp.LDOPF.om.pg.value

array([0.02000853, 0.0341217 , 0.0341217 , 0.02000853, 0.04000935,
       0.0341217 , 0.0341217 , 0.0341217 , 0.0341217 , 0.0341217 ,
       0.0341217 ])

In [25]:
# check lower limit of pg
ssp.LDOPF.pmin.v

array([0.02, 0.  , 0.  , 0.02, 0.04, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ])

In [26]:
# check upper limit of pg
ssp.LDOPF.pmax.v

array([2. , 0.2, 0.2, 2. , 2. , 0.2, 0.2, 0.2, 0.2, 0.2, 0.2])

In [28]:
M.value

array([0., 0., 0., 0., 0., 0., 0., 0.])

# 

In [29]:
D.value

array([0., 0., 0., 0., 0., 0., 0., 0.])